In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn import preprocessing

In [2]:
data = pd.read_csv('1429_1.csv')

c:\Users\saifalikhan\OneDrive - OPTIMAL STRATEGIX GROUP INC\Desktop\Monkey_learn\.monlvenv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data.columns

Index(['id', 'name', 'asins', 'brand', 'categories', 'keys', 'manufacturer',
       'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username'],
      dtype='object')

In [51]:
def sent(n):
    if n==5 or n==4:
        return 'Postive'
    elif n==3:
        return 'Nuetral'
    elif n==2 or n==1:
        return 'Negetive'
    else:
        pass

In [52]:
data_cols=data[['reviews.text','reviews.rating']]

In [53]:
data_cols['sentiments']=data_cols['reviews.rating'].apply(sent)

c:\Users\saifalikhan\OneDrive - OPTIMAL STRATEGIX GROUP INC\Desktop\Monkey_learn\.monlvenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
data_cols['sentiments'].value_counts()

Postive     32316
Nuetral      1499
Negetive      812
Name: sentiments, dtype: int64

In [55]:
data_cols_pos=data_cols[(data_cols['sentiments']=='Postive')][:812]
data_cols_neu=data_cols[(data_cols['sentiments']=='Nuetral')][:812]
data_cols_neg=data_cols[(data_cols['sentiments']=='Negetive')][:812]
data_2=pd.concat([data_cols_pos,data_cols_neu,data_cols_neg])

In [56]:
data_2['reviews.text']=data_2['reviews.text'].astype(str)
data_2['reviews.text']=data_2['reviews.text'].str.lower()
data_2['token_text']=data_2['reviews.text'].apply(lambda x :sent_tokenize(x))

In [57]:
label_encoder = preprocessing.LabelEncoder()

In [58]:
data_2['sentiments_en']=label_encoder.fit_transform(data_2['sentiments'])

In [78]:
from model import InferSent
import torch

V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = r'C:\Users\saifalikhan\OneDrive - OPTIMAL STRATEGIX GROUP INC\Desktop\Monkey_learn\GloVe\glove.840B.300d.txt'
model.set_w2v_path(W2V_PATH)

In [79]:
sentences=data_2['reviews.text'].to_list()

In [80]:
sentences=''.join(sentences)

In [81]:
sentences=sent_tokenize(sentences)

In [82]:
model.build_vocab(sentences, tokenize=True)

Found 4894(/7059) words with w2v vectors
Vocab size : 4894


In [59]:
data_2

,reviews.text,reviews.rating,sentiments,token_text,sentiments_en
0,this product so far has not disappointed. my c...,5.0,Postive,"[this product so far has not disappointed., my...",2
1,great for beginner or experienced person. boug...,5.0,Postive,"[great for beginner or experienced person., bo...",2
2,inexpensive tablet for him to use and learn on...,5.0,Postive,[inexpensive tablet for him to use and learn o...,2
3,i've had my fire hd 8 two weeks now and i love...,4.0,Postive,[i've had my fire hd 8 two weeks now and i lov...,2
4,i bought this for my grand daughter when she c...,5.0,Postive,[i bought this for my grand daughter when she ...,2
...,...,...,...,...,...
34654,this is exactly like any other usb power charg...,1.0,Negetive,[this is exactly like any other usb power char...,0
34656,amazon should include this charger with the ki...,1.0,Negetive,[amazon should include this charger with the k...,0
34657,love my kindle fire but i am really disappoint...,1.0,Negetive,[love my kindle fire but i am really disappoin...,0
34658,i was surprised to find it did not come with a...,1.0,Negetive,[i was surprised to find it did not come with ...,0


In [84]:
data_2['text_vec']=data_2['reviews.text'].apply(lambda x : model.encode(x)[0])

c:\Users\saifalikhan\OneDrive - OPTIMAL STRATEGIX GROUP INC\Desktop\Monkey_learn\model.py:207: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentences = np.array(sentences)[idx_sort]


In [123]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [124]:
data_2['spacy_vector']=data_2['reviews.text'].apply(lambda x:nlp(x).vector)

In [125]:
data_2

,reviews.text,reviews.rating,sentiments,token_text,sentiments_en,text_vec,spacy_vector
0,this product so far has not disappointed. my c...,5.0,Postive,"[this product so far has not disappointed., my...",2,"[-0.028743807, 0.04943118, -0.16104074, -0.028...","[-0.25019306, 2.7163677, -4.024124, -1.5867251..."
1,great for beginner or experienced person. boug...,5.0,Postive,"[great for beginner or experienced person., bo...",2,"[-0.017154634, 0.04943118, -0.17540525, -0.037...","[-1.9230068, -0.6920327, -4.27986, -1.771333, ..."
2,inexpensive tablet for him to use and learn on...,5.0,Postive,[inexpensive tablet for him to use and learn o...,2,"[0.0081173545, 0.0701023, -0.14298782, -0.0370...","[-1.1673046, 1.7241348, -3.271428, -0.3905851,..."
3,i've had my fire hd 8 two weeks now and i love...,4.0,Postive,[i've had my fire hd 8 two weeks now and i lov...,2,"[0.0081173545, 0.0701023, -0.14298782, -0.0370...","[-0.9205441, 1.2842977, -3.5087426, -0.0970070..."
4,i bought this for my grand daughter when she c...,5.0,Postive,[i bought this for my grand daughter when she ...,2,"[0.0081173545, 0.0701023, -0.14298782, -0.0370...","[-0.79002047, 1.180209, -2.8549027, -0.8414615..."
...,...,...,...,...,...,...,...
34654,this is exactly like any other usb power charg...,1.0,Negetive,[this is exactly like any other usb power char...,0,"[-0.028743807, 0.04943118, -0.16104074, -0.028...","[-1.5329013, 0.6928197, -2.25504, 0.30614275, ..."
34656,amazon should include this charger with the ki...,1.0,Negetive,[amazon should include this charger with the k...,0,"[0.01666666, 0.04943118, -0.1544277, 0.0354411...","[-2.0224683, 1.3657181, -3.5353205, -0.2447792..."
34657,love my kindle fire but i am really disappoint...,1.0,Negetive,[love my kindle fire but i am really disappoin...,0,"[-0.017154632, 0.04943118, -0.17540525, -0.037...","[-0.81430537, 0.32322833, -2.7721627, -0.85136..."
34658,i was surprised to find it did not come with a...,1.0,Negetive,[i was surprised to find it did not come with ...,0,"[0.0081173545, 0.0701023, -0.14298782, -0.0370...","[-1.7938577, 1.2113273, -3.1553993, -2.3094058..."


In [161]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(
    data_2['text_vec'].values,
    data_2['sentiments_en'],
    test_size=0.2,
    random_state=2022
)

In [167]:
x_train, x_test, y_train, y_test = train_test_split(
data_2['spacy_vector'].values,
data_2['sentiments_en'],
test_size=0.2,
random_state=2022)


In [117]:
x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [162]:
x_train=np.stack(x_train)
x_test=np.stack(x_test)

In [153]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.transform(x_test)
clf=MultinomialNB()
clf.fit(x_train_scaled,y_train)

MultinomialNB()

In [149]:
y_pred=clf.predict(x_test_scaled)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.38      0.52      0.44       171
           1       0.24      0.22      0.23       145
           2       0.37      0.26      0.30       172

    accuracy                           0.34       488
   macro avg       0.33      0.33      0.32       488
weighted avg       0.33      0.34      0.33       488



In [154]:
y_pred=clf.predict(x_test_scaled)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.58      0.55      0.57       171
           1       0.40      0.50      0.45       145
           2       0.67      0.57      0.62       172

    accuracy                           0.54       488
   macro avg       0.55      0.54      0.54       488
weighted avg       0.56      0.54      0.55       488



In [155]:
from sklearn.ensemble import RandomForestClassifier
r_clf=RandomForestClassifier()
r_clf.fit(x_train_scaled,y_train)

RandomForestClassifier()

In [115]:
y_pred=r_clf.predict(x_test_scaled)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.37      0.46      0.41       171
           1       0.25      0.30      0.27       145
           2       0.38      0.23      0.29       172

    accuracy                           0.33       488
   macro avg       0.34      0.33      0.32       488
weighted avg       0.34      0.33      0.33       488



In [156]:
y_pred=r_clf.predict(x_test_scaled)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.66      0.64      0.65       171
           1       0.46      0.49      0.47       145
           2       0.66      0.63      0.65       172

    accuracy                           0.59       488
   macro avg       0.59      0.59      0.59       488
weighted avg       0.60      0.59      0.60       488



In [168]:
from sklearn.ensemble import GradientBoostingClassifier
bo_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
     max_depth=1, random_state=0).fit(x_train_scaled, y_train)

In [164]:
y_pred=bo_clf.predict(x_test_scaled)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.64      0.64      0.64       171
           1       0.47      0.52      0.50       145
           2       0.67      0.62      0.65       172

    accuracy                           0.60       488
   macro avg       0.60      0.59      0.60       488
weighted avg       0.60      0.60      0.60       488



In [169]:
y_pred=bo_clf.predict(x_test_scaled)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.64      0.64      0.64       171
           1       0.47      0.52      0.50       145
           2       0.67      0.62      0.65       172

    accuracy                           0.60       488
   macro avg       0.60      0.59      0.60       488
weighted avg       0.60      0.60      0.60       488

